# Grover's algorithm
Grover's algorithm is a search algorithm to find out the data efficiently from random data.

This algorithm is based on quantum amplitude amplification.

# Explanation of the algorithm

The algorithm iterates through quantum amplitude amplification to bring the initial state $\lvert s\rangle$ closer to the state we are looking for $\lvert \omega\rangle$.

$$
\lvert s \rangle = \cos\bigl(\frac{\theta}{2}\bigr) \tilde{\lvert s' \rangle} - \sin\bigl(\frac{\theta}{2}\bigr) \tilde{\lvert \omega \rangle}
$$

From the above equation, if we perform amplitude amplification once, we get

$$
U_s U_{\omega} \lvert s\rangle =  \cos\bigl(\frac{3}{2}\theta\bigr)\tilde{\lvert s' \rangle} + \sin\bigl(\frac{3}{2}\theta\bigr)\tilde{\lvert \omega \rangle}
$$

Therefore, if we perform amplitude amplification $n$ times, we get

$$
\cos\bigl(\frac{1}{2}\theta + n\theta \bigr)\tilde{\lvert s' \rangle} + \sin\bigl(\frac{1}{2}\theta + n\theta\bigr)\tilde{\lvert \omega \rangle} = \cos\biggl\{\bigl(\frac{1}{2} + n \bigr)\theta \biggr\} \tilde{\lvert s' \rangle} + \sin\biggl\{\bigl(\frac{1}{2} + n \bigr)\theta \biggr\}\tilde{\lvert \omega \rangle}
$$

As the amplitude is amplified, the initial state $\lvert s\rangle$ in the above figure becomes closer and closer to $\lvert \omega\rangle$.
In other words, the probability amplitude of $\omega$ goes to 1.
However, if you keep repeating the process, it will just keep rotating, so you need to think about the number of times you want to repeat it.

Since we want to set the probability amplitude of $\omega$ to 1,

$$
\sin^2\biggl\{\bigl(\frac{1}{2} + k \bigr)\theta \biggr\} < 1 \xrightarrow{\sin >0}\bigl(\frac{1}{2} + k \bigr)\theta < \frac{\pi}{2} \Leftrightarrow k<\frac{\pi}{2\theta} - \frac{1}{2}
$$

From the above, the number of times the amplitude is amplified is determined and we can measure $\lvert \omega\rangle$ with high probability.

In the example implementation of amplitude amplification, the measurement probability of $\lvert \omega \rangle$ was set to 1 with one amplitude amplification.  
In the initial state $\lvert s\rangle$, which is an equally superposition state of two qubits, $\theta = \frac{\pi}{3}$.  
In this case, we can see from the above equation that a single amplitude amplification results in $U_s U_{\omega}\lvert s\rangle = \tilde{\lvert \omega \rangle}$.  
Thus, we were effectively running Grover's algorithm.